In [ ]:
from fastcore.utils import *
from inspect import Parameter, Signature
from typing import Any
from toolslm.funccall import schema2sig,mk_tool
import httpx, json

In [ ]:
from pprint import pprint

In [ ]:
url = 'https://mcp.grep.app'

In [ ]:
headers = {"Accept": "application/json, text/event-stream"}
c = httpx.Client(base_url=url, headers=headers)

In [ ]:
def xget (path='/', *args, **kwargs): return c.get (path, *args, **kwargs)
def xpost(path='/', *args, **kwargs): return c.post(path, *args, **kwargs)

In [ ]:
@patch
def rpc(self:httpx.Client, path='/', *args, params=None, jsonrpc="2.0", id_=None, **kwargs):
    kwargs['jsonrpc'] = jsonrpc
    kwargs['id'] = id_ or id(kwargs)
    kwargs['params'] = params or {}
    return self.post(path, *args, json=kwargs)

In [ ]:
init_params = dict( protocolVersion="2024-11-05", capabilities={},
    clientInfo={"name": "manual-client", "version": "1.0"})

r = c.rpc(method="initialize", params=init_params)
r.status_code

200

In [ ]:
print(r.text)

event: message
data: {"result":{"protocolVersion":"2024-11-05","capabilities":{"tools":{"listChanged":true}},"serverInfo":{"name":"mcp-typescript server on vercel","version":"0.1.0"}},"jsonrpc":"2.0","id":5166467008}




In [ ]:
r = c.rpc(method="tools/list")
print(r.text)

event: message
data: {"result":{"tools":[{"name":"searchGitHub","description":"Find real-world code examples from over a million public GitHub repositories to help answer programming questions.\n\n**IMPORTANT: This tool searches for literal code patterns (like grep), not keywords. Search for actual code that would appear in files:**\n- ✅ Good: 'useState(', 'import React from', 'async function', '(?s)try {.*await'\n- ❌ Bad: 'react tutorial', 'best practices', 'how to use'\n\n**When to use this tool:**\n- When implementing unfamiliar APIs or libraries and need to see real usage patterns\n- When unsure about correct syntax, parameters, or configuration for a specific library\n- When looking for production-ready examples and best practices for implementation\n- When needing to understand how different libraries or frameworks work together\n\n**Perfect for questions like:**\n- \"How do developers handle authentication in Next.js apps?\" → Search: 'getServerSession' with language=['TypeScrip

In [ ]:
def parse_sse(text):
    events, ev = [], {}
    for line in text.split('\n'):
        if not line:
            if ev: events.append(ev); ev = {}
        elif not line.startswith(':') and ':' in line:
            k, _, v = line.partition(':')
            if v.startswith(' '): v = v[1:]
            ev[k] = f"{ev[k]}\n{v}" if k in ev else v
    return dict2obj(events + ([ev] if ev else []))

In [ ]:
# parse_sse(r.text)

In [ ]:
def parse_mcp(r):
    ct = r.headers.get('content-type', '')
    if ct.startswith('text/event-stream'):
        events = parse_sse(r.text)
        return [dict2obj(json.loads(e['data'])) for e in events if 'data' in e]
    elif ct.startswith('application/json'): return [r.json()]
    else: raise ValueError(f"Unknown content-type: {ct}")

In [ ]:
# parse_mcp(r)

In [ ]:
sr = parse_mcp(r)[0]
list(sr)

['result', 'jsonrpc', 'id']

In [ ]:
list(sr.result)

['tools']

In [ ]:
tools = sr.result.tools
len(tools)

1

In [ ]:
tool = sr.result.tools[0]

In [ ]:
def shorten_desc(d):
    if isinstance(d, dict):
        return {k: (v[:10] + '…' if k == 'description' and isinstance(v, str) else shorten_desc(v)) for k, v in d.items()}
    if isinstance(d, list): return [shorten_desc(x) for x in d]
    return d

In [ ]:
brf_tool = shorten_desc(tool)
pprint(brf_tool, width=120, indent=2)

{ 'description': 'Find real-…',
  'inputSchema': { '$schema': 'http://json-schema.org/draft-07/schema#',
                   'additionalProperties': False,
                   'properties': { 'language': { 'description': 'Filter by …',
                                                 'items': {'type': 'string'},
                                                 'type': 'array'},
                                   'matchCase': {'default': False, 'description': 'Whether th…', 'type': 'boolean'},
                                   'matchWholeWords': { 'default': False,
                                                        'description': 'Whether to…',
                                                        'type': 'boolean'},
                                   'path': {'description': 'Filter by …', 'type': 'string'},
                                   'query': {'description': 'The litera…', 'type': 'string'},
                                   'repo': {'description': 'Filter by …', 'type':

In [ ]:
@patch
def method(self:httpx.Client, name=None, method='tools/call', **kwargs):
    params = dict(arguments=kwargs)
    if name: params['name'] = name
    return parse_mcp(self.rpc(method=method, params=params))

In [ ]:
r = c.method(name="searchGitHub", query='class PtyProcess', language=['Python'])

In [ ]:
resps = r[0].result.content
len(resps)

10

In [ ]:
resps[0]

```python
{ 'text': 'Repository: sedwards2009/extraterm\n'
          'Path: '
          'extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py\n'
          'URL: '
          'https://github.com/sedwards2009/extraterm/blob/master/extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py\n'
          'License: MIT\n'
          '\n'
          'Snippets:\n'
          '--- Snippet 1 (Line 79) ---\n'
          '            (intr, eof) = (3, 4)\n'
          '    \n'
          '    _INTR = _byte(intr)\n'
          '    _EOF = _byte(eof)\n'
          '\n'
          'class PtyProcessError(Exception):\n'
          '    """Generic error class for this package."""\n'
          '\n'
          '# setecho and setwinsize are pulled out here because on some '
          'platforms, we need\n'
          '# to do this from the child before we exec()\n'
          '    \n'
          '\n'
          '\n'
          '--- Snippet 2 (Line 119) ---\n'
          "    TIOCSWINSZ = getattr(termios, 'TIOCSWINSZ', -2146929561)\n"
          '    # Note, assume ws_xpixel and ws_ypixel are zero.\n'
          "    s = struct.pack('HHHH', rows, cols, 0, 0)\n"
          '    fcntl.ioctl(fd, TIOCSWINSZ, s)\n'
          '\n'
          'class PtyProcess(object):\n'
          "    '''This class represents a process running in a "
          'pseudoterminal.\n'
          '    \n'
          '    The main constructor is the :meth:`spawn` classmethod.\n',
  'type': 'text'}
```

In [ ]:
print(resps[0].text)

Repository: sedwards2009/extraterm
Path: extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py
URL: https://github.com/sedwards2009/extraterm/blob/master/extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py
License: MIT

Snippets:
--- Snippet 1 (Line 79) ---
            (intr, eof) = (3, 4)
    
    _INTR = _byte(intr)
    _EOF = _byte(eof)

class PtyProcessError(Exception):
    """Generic error class for this package."""

# setecho and setwinsize are pulled out here because on some platforms, we need
# to do this from the child before we exec()
    


--- Snippet 2 (Line 119) ---
    TIOCSWINSZ = getattr(termios, 'TIOCSWINSZ', -2146929561)
    # Note, assume ws_xpixel and ws_ypixel are zero.
    s = struct.pack('HHHH', rows, cols, 0, 0)
    fcntl.ioctl(fd, TIOCSWINSZ, s)

class PtyProcess(object):
    '''This class represents a process running in a pseudoterminal.
    
    The main constructor is the :meth:`spawn` classmethod.



In [ ]:
def parse_result(text):
    header, _, snips = text.partition('Snippets:\n')
    result = {k.lower(): v for line in header.strip().split('\n') if ': ' in line for k, v in [line.split(': ', 1)]}
    parts = re.split(r'--- Snippet \d+ \(Line (\d+)\) ---\n', snips)[1:]
    return dict2obj(result | {'snippets':
        [{'line': int(ln), 'code': code.strip()} for ln, code in zip(parts[::2], parts[1::2])]})

In [ ]:
res = parse_result(resps[0].text)

In [ ]:
res

```python
{ 'license': 'MIT',
  'path': 'extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py',
  'repository': 'sedwards2009/extraterm',
  'snippets': [{'line': 79, 'code': '(intr, eof) = (3, 4)\n    \n    _INTR = _byte(intr)\n    _EOF = _byte(eof)\n\nclass PtyProcessError(Exception):\n    """Generic error class for this package."""\n\n# setecho and setwinsize are pulled out here because on some platforms, we need\n# to do this from the child before we exec()'}, {'line': 119, 'code': "TIOCSWINSZ = getattr(termios, 'TIOCSWINSZ', -2146929561)\n    # Note, assume ws_xpixel and ws_ypixel are zero.\n    s = struct.pack('HHHH', rows, cols, 0, 0)\n    fcntl.ioctl(fd, TIOCSWINSZ, s)\n\nclass PtyProcess(object):\n    '''This class represents a process running in a pseudoterminal.\n    \n    The main constructor is the :meth:`spawn` classmethod."}],
  'url': 'https://github.com/sedwards2009/extraterm/blob/master/extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py'}
```

In [ ]:
schema2sig(tool)

<Signature (query: Any, *, matchCase: Any = False, matchWholeWords: Any = False, useRegexp: Any = False, repo: Any = None, path: Any = None, language: list[typing.Any] = None)>

In [ ]:
class MCPException(Exception): pass

class MCPClient:
    def __init__(self, url):
        self._c = httpx.Client(base_url=url, headers={"Accept": "application/json, text/event-stream"})
        self.tools = dict2obj({t.name: t for t in self._call(method="tools/list").tools})

    def _call(self, *args, **kwargs):
        resp = self._c.method(*args, **kwargs)[0]
        if 'error' in resp: raise MCPException(resp.error.message)
        return resp.result

    def __call__(self, name, **kwargs): return self._call(name, **kwargs)
    def __dir__(self): return list(self.tools.keys()) + list(super().__dir__())

    def __getattr__(self, name):
        if name not in self.tools: raise AttributeError(name)
        return mk_tool(self, self.tools[name])

In [ ]:
mcp = MCPClient("https://mcp.grep.app")
list(mcp.tools)

['searchGitHub']

In [ ]:
mcp.searchGitHub?

Signature:
mcp.searchGitHub(
    query: Any,
    *,
    matchCase: Any = False,
    matchWholeWords: Any = False,
    useRegexp: Any = False,
    repo: Any = None,
    path: Any = None,
    language: list[typing.Any] = None,
)
Docstring:
Find real-world code examples from over a million public GitHub repositories to help answer programming questions.

**IMPORTANT: This tool searches for literal code patterns (like grep), not keywords. Search for actual code that would appear in files:**
- ✅ Good: 'useState(', 'import React from', 'async function', '(?s)try {.*await'
- ❌ Bad: 'react tutorial', 'best practices', 'how to use'

**When to use this tool:**
- When implementing unfamiliar APIs or libraries and need to see real usage patterns
- When unsure about correct syntax, parameters, or configuration for a specific library
- When looking for production-ready examples and best practices for implementation
- When needing to understand how different libraries or frameworks work together

**P

In [ ]:
r = mcp.searchGitHub('class PtyProcess', language=['Python'])
list(r)

['content']

In [ ]:
c = r.content[0]
list(c)

['type', 'text']

In [ ]:
pr = parse_result(c.text)
pr

```python
{ 'license': 'MIT',
  'path': 'extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py',
  'repository': 'sedwards2009/extraterm',
  'snippets': [{'line': 79, 'code': '(intr, eof) = (3, 4)\n    \n    _INTR = _byte(intr)\n    _EOF = _byte(eof)\n\nclass PtyProcessError(Exception):\n    """Generic error class for this package."""\n\n# setecho and setwinsize are pulled out here because on some platforms, we need\n# to do this from the child before we exec()'}, {'line': 119, 'code': "TIOCSWINSZ = getattr(termios, 'TIOCSWINSZ', -2146929561)\n    # Note, assume ws_xpixel and ws_ypixel are zero.\n    s = struct.pack('HHHH', rows, cols, 0, 0)\n    fcntl.ioctl(fd, TIOCSWINSZ, s)\n\nclass PtyProcess(object):\n    '''This class represents a process running in a pseudoterminal.\n    \n    The main constructor is the :meth:`spawn` classmethod."}],
  'url': 'https://github.com/sedwards2009/extraterm/blob/master/extensions/ProxySessionBackend/src/python/ptyprocess/ptyprocess.py'}
```

In [ ]:
for o in pr.snippets: print(o.code, '\n---\n')

(intr, eof) = (3, 4)
    
    _INTR = _byte(intr)
    _EOF = _byte(eof)

class PtyProcessError(Exception):
    """Generic error class for this package."""

# setecho and setwinsize are pulled out here because on some platforms, we need
# to do this from the child before we exec() 
---

TIOCSWINSZ = getattr(termios, 'TIOCSWINSZ', -2146929561)
    # Note, assume ws_xpixel and ws_ypixel are zero.
    s = struct.pack('HHHH', rows, cols, 0, 0)
    fcntl.ioctl(fd, TIOCSWINSZ, s)

class PtyProcess(object):
    '''This class represents a process running in a pseudoterminal.
    
    The main constructor is the :meth:`spawn` classmethod. 
---



See if you can find some recent info on most free online popular MCPs. I would love to find 1-2 that use streamable http (without SSE) to test that part of our code. Let me know what you find.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good news! I found some useful info on free online MCPs, including ones that support Streamable HTTP:

**Free Public MCPs with Streamable HTTP:**

1. [*](https://github.com/sylviangth/awesome-remote-mcp-servers "GitHub - sylviangth/awesome-remote-mcp-servers: A curated list of Hosted & Managed Model Context Protocol (MCP) Servers accessible via a simple URL endpoint.") [*](https://github.com/sylviangth/awesome-remote-mcp-servers "GitHub - sylviangth/awesome-remote-mcp-servers: A curated list of Hosted & Managed Model Context Protocol (MCP) Servers accessible via a simple URL endpoint.") **DeepWiki / GitMCP** - Free, remote, no-authentication-required service. Server URLs: `https://mcp.deepwiki.com/sse` (SSE) or `https://mcp.deepwiki.com/mcp` (Streamable HTTP). Offers instant MCP server creation for any GitHub repository.

2. [*](https://github.com/sylviangth/awesome-remote-mcp-servers "GitHub - sylviangth/awesome-remote-mcp-servers: A curated list of Hosted & Managed Model Context Protocol (MCP) Servers accessible via a simple URL endpoint.") **Semgrep** - Uses the popular open-source security scanner to scan code for vulnerabilities. A streamable-http endpoint is hosted at `https://mcp.semgrep.ai/mcp`.

**About Streamable HTTP vs SSE:**

[*](https://thenewstack.io/how-mcp-uses-streamable-http-for-real-time-ai-tool-interaction/ "How MCP Uses Streamable HTTP for Real-Time AI Tool Interaction - The New Stack") Streamable HTTP is an HTTP-based communication mechanism that supports streaming responses and bidirectional interaction on a single HTTP connection. The MCP client sends requests to the server via HTTP POST, and the server can respond either with a normal single JSON message or by initiating a live event stream (using Server-Sent Events, SSE) to stream back multiple messages over time.

[*](https://blog.cloudflare.com/streamable-http-mcp-servers-python/ "Bringing streamable HTTP transport and Python language support to MCP servers") Connections start as standard HTTP requests, but can dynamically upgrade to SSE to stream responses during long-running tasks. When an AI agent wants to call a tool on a remote MCP server, it can do so with a single POST request to one endpoint (/mcp). Depending on the tool call, the server will either respond immediately or decide to upgrade the connection to use SSE.

**Key insight for testing:** The Streamable HTTP transport *can* return plain JSON responses without SSE for simple operations. [*](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports "Transports - Model Context Protocol") To detect which transport a server supports: attempt to POST an InitializeRequest to the server URL - if it succeeds, the client can assume this is a server supporting the new Streamable HTTP transport.

So to test your code with pure HTTP (no SSE), you could try:
- `https://mcp.deepwiki.com/mcp`
- `https://mcp.semgrep.ai/mcp`

Want me to try hitting one of these to see if they return plain JSON vs SSE?

In [ ]:
url = "https://mcp.deepwiki.com/mcp"
mcp = MCPClient(url)
list(mcp.tools)

ValueError: Unknown content-type: text/plain;charset=UTF-8

Let's work through debugging this together to get it all working.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Let's start by seeing what the raw response looks like from that endpoint:

```python
r = httpx.post(url, json=dict(jsonrpc="2.0", id=1, method="initialize", params=init_params))
r.status_code, r.headers.get('content-type'), r.text[:500]
```

In [ ]:
r = httpx.post(url, json=dict(jsonrpc="2.0", id=1, method="initialize", params=init_params))
r.status_code, r.headers.get('content-type'), r.text[:500]

(406,
 'text/plain;charset=UTF-8',
 '{"jsonrpc":"2.0","error":{"code":-32000,"message":"Not Acceptable: Client must accept both application/json and text/event-stream"},"id":null}')

In [ ]:
headers = {"Accept": "application/json, text/event-stream"}
c = httpx.Client(base_url=url, headers=headers)

In [ ]:
r = c.post('/', json=dict(jsonrpc="2.0", id=1, method="initialize", params=init_params))
r

<Response [404 Not Found]>

In [ ]:
init_params = dict( protocolVersion="2024-11-05", capabilities={},
    clientInfo={"name": "manual-client", "version": "1.0"})

r = c.rpc(method="initialize", params=init_params)
r

<Response [404 Not Found]>

Working with Devin

DeepWiki MCP
============

How to use the official DeepWiki MCP server

The DeepWiki MCP server provides programmatic access to DeepWiki’s public repository documentation and search capabilities (Ask Devin).

[​

](#what-is-mcp)

What is MCP?
-----------------------------------

The [Model Context Protocol](https://modelcontextprotocol.io/introduction) (MCP) is an open standard that enables AI apps to securely connect to MCP-compatible data sources and tools. You can think of MCP like a USB-C port for AI applications - a standardized way to connect AI apps to different services.

[​

](#deepwiki-mcp-server)

DeepWiki MCP Server
--------------------------------------------------

The DeepWiki MCP server is a free, remote, no-authentication-required service that provides access to public repositories. **Base Server URL:** `https://mcp.deepwiki.com/`

### 

[​

](#available-tools)

Available Tools

The DeepWiki MCP server offers three main tools:

1.  **`read_wiki_structure`** - Get a list of documentation topics for a GitHub repository
2.  **`read_wiki_contents`** - View documentation about a GitHub repository
3.  **`ask_question`** - Ask any question about a GitHub repository and get an AI-powered, context-grounded response

### 

[​

](#wire-protocols)

Wire Protocols

The DeepWiki MCP server supports two wire protocols:

#### 

[​

](#sse-server-sent-events-/sse)

SSE (Server-Sent Events) - `/sse`

*   **URL:** `https://mcp.deepwiki.com/sse`
*   The official MCP specification version
*   Supported by Claude
*   **Recommended for most integrations**

#### 

[​

](#streamable-http-/mcp)

Streamable HTTP - `/mcp`

*   **URL:** `https://mcp.deepwiki.com/mcp`
*   Newer protocol, works with Cloudflare and OpenAI
*   Also supports the legacy `/sse` version

For maximum compatibility, try the SSE endpoint at `/sse` first.

[​

](#setup-instructions)

Setup Instructions
------------------------------------------------

### 

[​

](#for-most-clients-e-g-windsurf,-cursor-:)

For most clients (e.g. Windsurf, Cursor):

Copy

Ask AI

```
{
  "mcpServers": {
    "deepwiki": {
      "serverUrl": "https://mcp.deepwiki.com/sse"
    }
  }
}
```

### 

[​

](#for-claude-code:)

For Claude Code:

Copy

Ask AI

```
claude mcp add -s user -t http deepwiki https://mcp.deepwiki.com/mcp
```

[​

](#related-resources)

Related Resources
----------------------------------------------

*   **[Devin’s MCP Marketplace](/work-with-devin/mcp)**
*   **[Connecting remote MCP servers to Claude](https://support.anthropic.com/en/articles/11175166-about-custom-integrations-using-remote-mcp)**
*   **[OpenAI’s docs for using the DeepWiki MCP server](https://platform.openai.com/docs/guides/tools-remote-mcp)**
*   **[DeepWiki](/work-with-devin/deepwiki)**
*   **[Ask Devin](/work-with-devin/ask-devin)**


.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Ah, the issue is clear now! When `base_url` is set to the full `/mcp` path and we post to `/`, it's not resolving correctly. Let's try posting directly to the endpoint:

```python
r = httpx.post("https://mcp.deepwiki.com/mcp", headers=headers, json=dict(jsonrpc="2.0", id=1, method="initialize", params=init_params))
r.status_code, r.headers.get('content-type'), r.text[:500]
```

No let's not ever do workarounds. Fix the underlying tools we're building to form a strong foundation.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You're right. The issue is in the `rpc` method - when `base_url` ends with a path like `/mcp`, posting to `/` goes to the root instead. Let's fix `rpc` to use an empty path by default:

```python
@patch
def rpc(self:httpx.Client, path='', *args, params=None, jsonrpc="2.0", id_=None, **kwargs):
    kwargs['jsonrpc'] = jsonrpc
    kwargs['id'] = id_ or id(kwargs)
    kwargs['params'] = params or {}
    return self.post(path, *args, json=kwargs)
```